<a href="https://colab.research.google.com/github/Sompote/RIC_predict/blob/main/FFN_RIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Transformer Autoencoder to predict RIC combind model by Dr. Sompote Youwai 2023
import numpy as np
import pandas as pd

# Import libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd



df = pd.read_excel('/content/drive/MyDrive/RIC/RIC3aug_revise.xlsx')
data=np.array(df)
X=data[:,1:29]
Y=data[:,34:62]
x_feature=data[:,29:32]




In [17]:
X_data_com=np.concatenate((X,x_feature),axis=1)
X_train=X
Y_train=Y
sc_y=StandardScaler()
sc = StandardScaler()
X_feature=x_feature
X_train=X

x_train = sc.fit_transform(X_train)
y_train=sc_y.fit_transform(Y_train)


In [18]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train, random_state=1,test_size=0.05)

In [25]:
#Model generation part
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, Flatten, concatenate,Reshape, Dropout,Bidirectional
from keras.layers import MaxPooling1D
from keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanAbsolutePercentageError

def FNN_dense(inputdim,outputdim):
  # Create the CNN model
  input = Input(shape=inputdim,name='dense_input')
  layer = Dense(100, activation='sigmoid')(input)
  layer = Dense(50, activation='sigmoid')(layer)
  output= Dense(outputdim, activation='linear')(layer)


  model = Model(inputs=input, outputs=output,name='FNN_model')
  return model

FNN=FNN_dense(X_train.shape[1],Y_train.shape[1])

FNN.compile(optimizer='adam', loss='mse', metrics=['mae','mape'])
FNN.summary()
history=FNN.fit(X_train, Y_train, epochs=2000, batch_size=10,validation_data=(X_test, Y_test))



Model: "FNN_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (InputLayer)    [(None, 28)]              0         
                                                                 
 dense_18 (Dense)            (None, 100)               2900      
                                                                 
 dense_19 (Dense)            (None, 50)                5050      
                                                                 
 dense_20 (Dense)            (None, 28)                1428      
                                                                 
Total params: 9,378
Trainable params: 9,378
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
3/3 [==============================] - 1s 102ms/step - loss: 1.2882 - mae: 0.8881 - mape: 781.5949 - val_loss: 1.6831 - val_mae: 1.0007 - val_mape: 105.2771
Epoch 2/2000
3/3 

In [26]:
def mape(y_true, y_pred):
  mape=np.mean(np.abs((Y - y_pred)/Y))*100
  # return mean of percentage errors
  return mape

def mae(y_true, y_pred):
  mae=np.mean(np.abs(Y - y_pred))
  # return mean of percentage errors
  return mae

y=FNN.predict(x_train)
y_pred=sc_y.inverse_transform(y)
mape=mape(Y,y_pred)
mae=mae(Y,y_pred)

print('MAPE=', mape)
print('MAPE=', mae)



1/1 [==============================] - 0s 64ms/step
MAPE= 12.13181237607263
MAPE= 0.40738827848340226
